In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import random
import joblib
import os
os.chdir('Resources/')

from custom_tuple_scaler import CustomTupleScaler

In [2]:
df = pd.read_csv('6_CC_Encrypted_Data.csv')

In [3]:
Y = df[['HeartDisease']]
Y

,HeartDisease
0,0
1,1
2,0
3,1
4,0
...,...
3245,1
3246,1
3247,0
3248,0


In [4]:
c1_list = []
c2_list = []

for k in range(11):
    cnt = 0
    c1_sum = 0
    c2_sum = 0
    
    messages = df.iloc[:, k].values

    for i, message in enumerate(messages, 1):
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))
            c1_sum += c1
            c2_sum += c2
            cnt += 1
    
    print(c1_sum//cnt, ",", c2_sum//cnt)

    c1_list.append(c1_sum//cnt)
    c2_list.append(c2_sum//cnt)

print(c1_list)
print(c2_list)

6965361356960251970681699416527167003396930684399 , 30217139673302031254829592992378024533878940689657
54600078953351133152003341118517817314167801601939 , 23882417530828833820232030289434178573895492614734
36649207273954309735937122332365840930023873680884 , 31257445741747527543331712533634722199901749334179
83860246762706255172761784370775066461479861118299 , 47736326788307872921021879683387002044728535726667
16043955655060611551799098102694158622740324518298 , 43570977182708344300493988886736242425372723276623
48796268997449217309075469418552965717916389030766 , 21270487134087081655449550974215752004786739471885
3857767734957454709824171872090418722115974714484 , 6007744126266429215264072859610897495757862371132
83297904217885534795359347609074572793018462457138 , 40148136876669348027076612145139127910533037019312
12956688826076395746654158514081595251862400502558 , 5473654934117652083434044599580847510549404051408
5759314526113684676869960964229505210431962579413 , 9488133877349612

In [5]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

In [6]:
imputed_data = pd.DataFrame()

for k in range(11):
    
    messages = df.iloc[:, k].values

    imputed_column = []

    for i, message in enumerate(messages, 1):
        if message is None:
            c1 = c1_list[k]
            c2 = c2_list[k]
        
        if message is not None and message == message:
            c1, c2 = map(int, message.strip("()").split(","))

        imputed_column.append(f"({c1},{c2})")

    imputed_data[column_names[k]] = imputed_column

In [7]:
scaler = joblib.load('13_CC_Scaler.joblib')

In [8]:
df_scaled = scaler.transform(imputed_data)
df_scaled = pd.DataFrame(df_scaled, columns=imputed_data.columns)
# df_scaled.to_csv('scaled_new_data.csv', index=False)

g:\Thesis - Copy\Resources\custom_tuple_scaler.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  c2_values = X.applymap(self._extract_c2)


In [9]:
with open("12_SP_Test_Data_RF_IF.txt", "r") as file:
    test_indices = eval(file.read()) 

In [10]:
file_path = "13_CC_F1_Scores_RF_IF.txt"

with open(file_path, "r") as f:
    lines = f.readlines()

reference_f1_scores = []
for line in lines:
    if "F1-score for fold" in line:
        score = float(line.split(":")[1].strip())
        reference_f1_scores.append(score)

In [11]:
fold_to_test = random.randint(1, 5)

test_indices_fold = test_indices[fold_to_test - 1] 
print(f"Test indices for fold {fold_to_test}: {test_indices_fold}")

X_test = df_scaled.iloc[test_indices_fold]

model_path = f'12_SP_Model_RF_IF_{fold_to_test}.joblib'
model = joblib.load(model_path)

Y_test = Y.iloc[test_indices_fold]
Y_pred = model.predict(X_test)

computed_f1_fold = f1_score(Y_test, Y_pred) * 100

reference_f1_fold = reference_f1_scores[fold_to_test - 1]

print(f"Computed F1-score for fold 1: {computed_f1_fold:.2f}")
print(f"Reference F1-score for fold 1: {reference_f1_fold:.2f}")

if np.isclose(computed_f1_fold, reference_f1_fold, atol=1e-2):
    print("The computed F1-score matches the reference score.")
else:
    print("The computed F1-score does not match the reference score.")

Test indices for fold 1: [0, 3, 8, 13, 21, 22, 25, 26, 27, 30, 31, 39, 51, 53, 55, 58, 66, 72, 78, 81, 84, 86, 87, 89, 98, 99, 105, 118, 123, 139, 141, 144, 152, 162, 164, 172, 175, 176, 184, 192, 195, 196, 203, 213, 224, 228, 234, 237, 243, 249, 253, 254, 261, 262, 264, 271, 275, 299, 302, 304, 314, 315, 316, 319, 321, 325, 326, 329, 330, 340, 342, 344, 346, 351, 356, 361, 366, 368, 371, 375, 390, 391, 393, 403, 415, 420, 422, 423, 437, 444, 445, 449, 464, 466, 469, 471, 473, 491, 494, 496, 498, 502, 512, 524, 534, 535, 542, 543, 546, 550, 555, 565, 566, 572, 575, 581, 590, 591, 597, 599, 607, 617, 638, 640, 642, 646, 651, 656, 664, 672, 673, 675, 681, 689, 690, 692, 695, 699, 708, 709, 712, 713, 716, 718, 720, 723, 732, 733, 734, 735, 741, 754, 756, 760, 771, 772, 785, 788, 796, 797, 800, 809, 812, 822, 839, 840, 851, 853, 856, 869, 877, 881, 885, 897, 915, 918, 923, 928, 932, 936, 937, 939, 943, 944, 946, 948, 950, 953, 956, 958, 964, 970, 971, 975, 985, 986, 989, 990, 993, 995, 996